In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
data = load_iris()

In [ ]:
X, y = data.data, data.target

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=101)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=101)

# **Train Base Models**

In [48]:
knn = KNeighborsClassifier(n_neighbors=5)

In [49]:
dt = DecisionTreeClassifier(random_state=101)

In [50]:
lr = LogisticRegression(max_iter=1000, random_state=101)

In [51]:
knn.fit(X_train, y_train)
dt.fit(X_train, y_train)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=101)

In [52]:
# Validation accuracy
knn_val_score = accuracy_score(y_val, knn.predict(X_val))
dt_val_score = accuracy_score(y_val, dt.predict(X_val))
lr_val_score = accuracy_score(y_val, lr.predict(X_val))

In [53]:
knn_val_score

1.0

In [54]:
dt_val_score

1.0

In [55]:
lr_val_score

1.0

## **Static Method**
Select the best model based on validation accuracy

In [56]:
if knn_val_score >= dt_val_score and knn_val_score >= lr_val_score:
    top_model = knn
elif dt_val_score >= knn_val_score and dt_val_score >= lr_val_score:
    top_model = dt
else:
    top_model = lr

In [57]:
top_model

KNeighborsClassifier()

In [58]:
static_predictions = top_model.predict(X_test)

In [59]:
static_predictions

array([1, 1, 1, 0, 0, 2, 1, 1, 1, 0, 2, 2, 0, 2, 1])

In [60]:
static_accuracy = accuracy_score(y_test, static_predictions)

In [61]:
static_accuracy

1.0

## **KNORA-Based Dynamic Method**

In [62]:
def knora_dynamic_selection(X_train, y_train, X_test, y_test, models, k=5):

    nn = NearestNeighbors(n_neighbors=k).fit(X_train)
    selected_preds = []

    for i, x in enumerate(X_test):

        # Find k-nearest neighbors of the current test instance
        distances, indices = nn.kneighbors([x])
        local_y = y_train[indices[0]]

        # Evaluate each model on the k-nearest neighbors
        local_scores = []

        for model in models:
            preds = model.predict(X_train[indices[0]])
            score = accuracy_score(local_y, preds)
            local_scores.append(score)

        # Select the model with the highest score
        best_model_index = np.argmax(local_scores)
        best_model = models[best_model_index]

        # Predict using the selected model
        selected_preds.append(best_model.predict([x])[0])

    return np.array(selected_preds)

In [63]:
models = [knn, dt, lr]

In [64]:
dynamic_predictions = knora_dynamic_selection(X_train, y_train, X_test, y_test, models, k=5)

In [65]:
dynamic_accuracy = accuracy_score(y_test, dynamic_predictions)

In [66]:
dynamic_accuracy

1.0